In [1]:
# Import libraries and packages for the project 
# import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup#pulling data out of HTML files
from time import sleep#slow down the execution to pass the security check
import csv
import pandas as pd
import numpy as np
import math
import re
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'/Users/jchen056/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/109/chromedriver'

In [2]:
df=pd.read_csv('data/CUNY_students.csv')
df['Name']=df['FName']+' '+df['LName']
df.dropna(inplace=True)


In [3]:
# df['Name']=df['Name']+df['Campus']

names=[]

for i in list(df['Name']):
    cur=i.replace(u'\xa0', u' ')
    names.append(cur)
print(sorted(names))
print(len(names))
print(len(set(names)))



[' Andrew  Blake ', ' Huma  Gulnar ', ' Ivan  Baez ', ' Nkem  Stanley ', ' Son  Seung ', 'Adashima  Oyo ', 'Adjiwanou  Gbagba ', 'Ainsley   Parkinson  ', 'Alice  Warren ', 'Alicia  Orellana ', 'Amarachukwu  Enemuo ', 'Amarilis   Bueno ', 'Ana  Alas ', 'Angela  Padilla ', 'Anthony  Cruz-Balbardy ', 'Beatriz   Salazar ', 'Beatriz   Salazar ', 'Beresford  \u202fKurton ', 'Carol   Woodburn-Mc Donald  ', 'Cathy   Ho ', 'Cathy  Ho ', 'Clara  Niet-Wire ', 'Clyde   Lewis ', 'David   Clarke ', 'Desire  Miessein  ', 'Diomaris   Padilla ', 'Dorean   Flores ', 'Femi   Ariyo ', 'Francoise  Sidime      ', 'Gillian   Cain ', 'Giselle   Acco ', 'Giselle  Acco ', 'Huma   Gulnar ', 'Irena  Jmukhadze ', 'Ivette   Beckles ', 'Jason  Mighty ', 'Jean-Marc  Carre  ', 'Jenny  Paredes ', 'Jo Ann   Caban ', 'Jodi-Ann   Young ', 'Jodi-Ann  Young ', 'Jonathan  Blaize ', 'Juliet  Baidoo ', 'Katrina   Cruz   ', 'Kelly  Levano ', 'Kelly  Levano ', 'Kirby   Brown ', 'Lina   Loaiza  ', 'Loni   Collins ', 'Lyndon   Hay

In [4]:
# names_modified=[]
# for i in names:
#     names_modified.append("".join(re.split("[^a-zA-Z]*", i)))
# # names_modified
names_final=list(set(names))
names_final=sorted(names_final)
len(names_final)
names_final

[' Andrew  Blake ',
 ' Huma  Gulnar ',
 ' Ivan  Baez ',
 ' Nkem  Stanley ',
 ' Son  Seung ',
 'Adashima  Oyo ',
 'Adjiwanou  Gbagba ',
 'Ainsley   Parkinson  ',
 'Alice  Warren ',
 'Alicia  Orellana ',
 'Amarachukwu  Enemuo ',
 'Amarilis   Bueno ',
 'Ana  Alas ',
 'Angela  Padilla ',
 'Anthony  Cruz-Balbardy ',
 'Beatriz   Salazar ',
 'Beresford  \u202fKurton ',
 'Carol   Woodburn-Mc Donald  ',
 'Cathy   Ho ',
 'Cathy  Ho ',
 'Clara  Niet-Wire ',
 'Clyde   Lewis ',
 'David   Clarke ',
 'Desire  Miessein  ',
 'Diomaris   Padilla ',
 'Dorean   Flores ',
 'Femi   Ariyo ',
 'Francoise  Sidime      ',
 'Gillian   Cain ',
 'Giselle   Acco ',
 'Giselle  Acco ',
 'Huma   Gulnar ',
 'Irena  Jmukhadze ',
 'Ivette   Beckles ',
 'Jason  Mighty ',
 'Jean-Marc  Carre  ',
 'Jenny  Paredes ',
 'Jo Ann   Caban ',
 'Jodi-Ann   Young ',
 'Jodi-Ann  Young ',
 'Jonathan  Blaize ',
 'Juliet  Baidoo ',
 'Katrina   Cruz   ',
 'Kelly  Levano ',
 'Kirby   Brown ',
 'Lina   Loaiza  ',
 'Loni   Collins ',
 'Lyndo

In [4]:
def finding_name(text):
    new_text=''
    for i, letter in enumerate(text):
        if i==0:
            pass
        if i and letter.isupper():
            new_text += ' '

        new_text += letter
    return new_text
names_final=[]
for i in names_modified:
    names_final.append(finding_name(i))
# names_final

NameError: name 'names_modified' is not defined

In [63]:
df['Name']=names_final
df['Name'].value_counts().head(10)

Minerva Francis    2
Giselle Acco       2
Beatriz Salazar    2
Jodi Ann Young     2
Magalie Bruneus    2
Huma Gulnar        2
Sefton Bennett     2
Kelly Levano       2
Cathy Ho           2
Samira Daher       2
Name: Name, dtype: int64

In [65]:
df[df['Name'].isin(['Minerva Francis','Giselle Acco','Beatriz Salazar','Jodi Ann Young','Magalie Bruneus'])]

,LName,FName,Campus,Name
8,Acco,Giselle,BMCC,Giselle Acco
53,Acco,Giselle,LGCC,Giselle Acco
65,Francis,Minerva,NYCCT,Minerva Francis
66,Salazar,Beatriz,NYCCT,Beatriz Salazar
68,Young,Jodi-Ann,NYCCT,Jodi Ann Young
71,Francis,Minerva,NYCCT,Minerva Francis
72,Salazar,Beatriz,NYCCT,Beatriz Salazar
73,Young,Jodi-Ann,NYCCT,Jodi Ann Young
75,Bruneus,Magalie,QCC,Magalie Bruneus
84,Bruneus,Magalie,QCC,Magalie Bruneus


In [52]:
names_final=list(set(names_final))
len(names_final)

82

good Youtube videos:
1. [link]https://www.youtube.com/watch?v=zkfLAY2OrtI
2. [link]https://www.linkedin.com/learning/python-automation-and-testing/what-is-the-html-dom-structure?autoplay=true&resume=false

In [2]:
# Task 1: Login to Linkedin

credential=open('login_credential.txt')
line=credential.readlines()
username=line[0]
password=line[1]

# Task 1.1: Open Chrome and Access Linkedin login site
driver =webdriver.Chrome()
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(2)

# Task 1.2: Key in login credentials
email_field = driver.find_element('name','session_key')
email_field.send_keys(username)
sleep(2)

password_field = driver.find_element('id','password')
password_field.send_keys(password)
sleep(2)

# Task 1.2: Click the Login button

login_field = driver.find_element('xpath',"/html/body/div/main/div[2]/div[1]/form/div[3]/button")
login_field.click()
sleep(2)

In [6]:
# Task 2: Search for the profile we want to crawl

# Task 2.1: Locate the search bar element
# search_field = driver.find_element('xpath',"//button[@aria-label='Click to start a search'][@type='button']")
# search_field.click()

# Task 2.2: Input the search query to the search bar
#search_query = input("what do you want to search:")#using the list from Dr C
type_field=driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input")
type_field.send_keys(names_final[0])

# Task 2.3: Search for CUNY student and click on People
type_field.send_keys(Keys.RETURN)
sleep(1)

driver.find_element('xpath','//*[@id="search-reusables__filters-bar"]/ul/li[1]/button').click()
driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input").clear()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="search-reusables__filters-bar"]/ul/li[1]/button"}
  (Session info: chrome=109.0.5414.119)
Stacktrace:
0   chromedriver                        0x00000001004fa7a8 chromedriver + 4896680
1   chromedriver                        0x0000000100479db3 chromedriver + 4369843
2   chromedriver                        0x00000001000c3a37 chromedriver + 477751
3   chromedriver                        0x000000010010862c chromedriver + 759340
4   chromedriver                        0x0000000100108881 chromedriver + 759937
5   chromedriver                        0x000000010014c054 chromedriver + 1036372
6   chromedriver                        0x000000010012e48d chromedriver + 914573
7   chromedriver                        0x000000010014944e chromedriver + 1025102
8   chromedriver                        0x000000010012e233 chromedriver + 913971
9   chromedriver                        0x00000001

In [7]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('div',class_="entity-result__item") 


    all_profile_url=[]
    for profile in profiles:
        profile_url=profile.find('a',class_="app-aware-link")['href']
        if profile_url not in all_profile_url:
            all_profile_url.append(profile_url)
    if len(all_profile_url)>=5:
        return all_profile_url[0:5]
    return all_profile_url


   


# # Task 3.2: Navigate through the next pages, and extract the profile URLs of each page

# def GetURL_pages():
#     input_page = 1
#     URLs_all_page = []
#     for page in range(input_page):
#         URLs_one_page = GetURL()
#         driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')#we need to scroll down to Next page button
#         sleep(1)
#         next_button = driver.find_element('xpath',"//button[@aria-label='Next'][@type='button']").click()
#         URLs_all_page = URLs_all_page + URLs_one_page
        
#         sleep(1)
#     return URLs_all_page

# URLs_all_page=GetURL_pages()
# print(URLs_all_page)

In [47]:
all_URLs=[]
all_URLs=np.concatenate((all_URLs,GetURL()),axis=None)
all_URLs

array(['https://www.linkedin.com/in/abdelhamid-jnane-01011b26?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAVbR0wBFW2fk0kw2QXU1AdVcx6QXmqrA6o'],
      dtype='<U136')

In [48]:
for i in range(1,len(names_final)):
    #step1: let us search for the person
    type_field=driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input")
    type_field.send_keys(names_final[i])
    type_field.send_keys(Keys.RETURN)
    sleep(2)
    all_URLs=np.concatenate((all_URLs,GetURL()),axis=None)
    sleep(1)
    driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input").clear()
    sleep(1)



In [51]:
URLs_all_page=[]

for i in all_URLs:
    if "https://www.linkedin.com/in/" in i:
        URLs_all_page.append(i)
# print(URLs_all_page)
len(URLs_all_page)

308

In [53]:
# dictionary of lists  
dict = {'url': URLs_all_page}  
       
df_url= pd.DataFrame(dict) 
df_url
# saving the dataframe 
df_url.to_csv('new_URL.csv') 


In [7]:
df_new=pd.read_csv('new_URL.csv')
df_new
# URLs=list(df_new['url'][260:])
#URLs.append(df_new['url'][297])
URLs=[
'https://www.linkedin.com/in/lori-collins-a6411b1b9',
'https://www.linkedin.com/in/vaughn-clai-marsh-a050932',
'https://www.linkedin.com/in/allieorellana',
'https://www.linkedin.com/in/aalasiglesias',
'https://www.linkedin.com/in/angela-padilla-985761237',
'https://www.linkedin.com/in/doreanflores',
'https://www.linkedin.com/in/anthony-cruz-balberdy-48815b73',
'https://www.linkedin.com/in/andrew-wills-67a1a289']

In [8]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file
with open('l1_missingPPL.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location','Degrees','Schools','Work Places','Work Titles','URL',]
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    
    for linkedin_URL in URLs:
        driver.get(linkedin_URL)
#         print(linkedin_URL)
        sleep(1)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'mt2 relative'})

        name = info_div.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').get_text().strip() #Remove unnecessary characters 
        #print('--- Profile name is: ', name)
        location = info_div.find('span', class_='text-body-small inline t-black--light break-words').get_text().strip() #Remove unnecessary characters 
        #print('--- Profile location is: ', location)
        title = info_div.find('div', class_='text-body-medium break-words').get_text().strip()
        #print('--- Profile title is: ', title)
        
        working_titles=[]
        working_places=[]
        if page_source.find('div',{'id':'experience'}):
            work_exp=page_source.find('div',{'id':'experience'}).find_parent().find('ul',{'class':'pvs-list ph5 display-flex flex-row flex-wrap'})
            for i in work_exp.find_all('span',class_='mr1 t-bold'):
                working_titles.append(i.get_text())
            for i in work_exp.find_all('span',class_='t-14 t-normal'):
                working_places.append(i.get_text())
        
        
        degrees=[]
        schools=[]
        if page_source.find('div',{'id':'education'}):
            ed_exp=page_source.find('div',{'id':'education'}).find_parent().find('ul',{'class':'pvs-list ph5 display-flex flex-row flex-wrap'})
            for i in ed_exp.find_all('span',class_='t-14 t-normal'):
                degrees.append(i.get_text())
            for j in ed_exp.find_all('span',class_='mr1 hoverable-link-text t-bold'):
                schools.append(j.get_text())
        
        writer.writerow({headers[0]:name, headers[2]:location, headers[1]:title,
                         headers[3]:degrees, headers[4]:schools,
                         headers[5]:working_places,headers[6]:working_titles, headers[7]:linkedin_URL})
        print('\n')
    
        



In [23]:
driver.get('https://www.linkedin.com/in/yisa-rumala-15001b16?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAM0vAwBvI3QP9veYd4C4oUxBoOAjt2Hua0')
sleep(1)
page_source = BeautifulSoup(driver.page_source, "html.parser")

working_titles=[]
working_places=[]
yrs_exp=[]
if page_source.find('div',{'id':'experience'}):
    work_exp=page_source.find('div',{'id':'experience'}).find_parent().find('ul',{'class':'pvs-list ph5 display-flex flex-row flex-wrap'})

    
    for i in work_exp.find_all('span',class_='mr1 t-bold'):
        working_titles.append(i.get_text())
    
    
    for i in work_exp.find_all('span',class_='t-14 t-normal'):
        working_places.append(i.get_text())


    for i in work_exp.find_all('span',class_='t-14 t-normal t-black--light'):
        yrs_exp.append(i.get_text())


print(working_titles)
print(working_places)
print(yrs_exp)

degrees=[]
schools=[]
years_ed=[]
if page_source.find('div',{'id':'education'}):
    ed_exp=page_source.find('div',{'id':'education'}).find_parent().find('ul',{'class':'pvs-list ph5 display-flex flex-row flex-wrap'})
    for i in ed_exp.find_all('span',class_='t-14 t-normal'):
        degrees.append(i.get_text())
    for j in ed_exp.find_all('span',class_='mr1 hoverable-link-text t-bold'):
        schools.append(j.get_text())
    for k in ed_exp.find_all('span',class_='t-14 t-normal t-black--light'):
        years_ed.append(k.get_text())
        
print(degrees)
print(schools)
print(years_ed)

['\nVisiting Research ScholarVisiting Research Scholar\n', '\nResearch Associate - Substitute Research Associate for 1 year then Full Research Associate Research Associate - Substitute Research Associate for 1 year then Full Research Associate \n', '\nPostdoctoral Visiting Research ScientistPostdoctoral Visiting Research Scientist\n']
['\nFull-time · 7 yrsFull-time · 7 yrs\n', '\nUniversity of Michigan, Ann Arbor - Randall LaboratoryUniversity of Michigan, Ann Arbor - Randall Laboratory\n', '\nInstitute for Ultrafast Spectroscopy and Lasers (IUSL), City University of New York · Full-timeInstitute for Ultrafast Spectroscopy and Lasers (IUSL), City University of New York · Full-time\n', '\nThe City University of New YorkThe City University of New York\n', '\n5 yrs 8 mos5 yrs 8 mos\n']
['\nDec 2020 - Present · 2 yrs 3 mosDec 2020 - Present · 2 yrs 3 mos\n', '\nUnited StatesUnited States\n', '\nMar 2016 - Dec 2020 · 4 yrs 10 mosMar 2016 - Dec 2020 · 4 yrs 10 mos\n', '\nEast Hartford, CTEas

In [17]:
page_source.find('div',{'id':'experience'})

In [29]:
df_o4=pd.read_csv('output4.csv')
df_o3=pd.read_csv('output3.csv')
df_o2=pd.read_csv('output2.csv')
df_o1=pd.read_csv('output1.csv')
df_o=pd.concat([df_o3,df_o2,df_o1,df_o4])
df_o.reset_index(drop=True,inplace=True)

In [27]:
campuses=['CUNY New York City College of Technology','College of Staten Island',
          'Lehman College','LaGuardia Community College',
          
          'Baruch College', 'Queens College',
           'Queensborough Community College',
          
          'Borough of Manhattan Community College','CUNY Bronx Community College',
          'Kingsborough Community College',
          
          'CUNY York College',
          'CUNY Hostos Community College', 'Hunter College',
          'Medgar Evers College, CUNY',
          'John Jay College (CUNY)',
         'Brooklyn College',
          'The City College of New York',]

reg_campus=''
for i in campuses:
    reg_campus+=i
    reg_campus+='|'
reg_campus=reg_campus[0:-1]
reg_campus

'CUNY New York City College of Technology|College of Staten Island|Lehman College|LaGuardia Community College|Baruch College|Queens College|Queensborough Community College|Borough of Manhattan Community College|CUNY Bronx Community College|Kingsborough Community College|CUNY York College|CUNY Hostos Community College|Hunter College|Medgar Evers College, CUNY|John Jay College (CUNY)|Brooklyn College|The City College of New York'

In [37]:
df_o['CUNY students']=df_o['Schools'].str.contains(reg_campus,regex=True)
CUNY_students=df_o[df_o['CUNY students']==True]
CUNY_students['Name'].value_counts()

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_32653/3692875245.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_o['CUNY students']=df_o['Schools'].str.contains(reg_campus,regex=True)


Mohammed Uddin                         2
Minerva Francis                        1
Vadim Acosta                           1
Professor Rasheem Northington          1
Ivan Baez, P.E.                        1
JoAnn Caban                            1
Natalie Dastas                         1
Juliet Baidoo                          1
Dr. Carol Woodburn McDonald            1
Uchenna Unachukwu, Ph.D                1
Nadine Bennett                         1
Jodi-Ann Young, MSc., ACSM             1
Rita Barley                            1
Onyekwere O.                           1
Sharon Lall-Ramnarine                  1
Jean-Marc Carre                        1
Mariya Alexandra Mayer, MSPAS, PA-C    1
Marvin Bennett                         1
Jenny Paredes, PhD                     1
Jason Mighty                           1
Moody Saint Louis                      1
Françoise S.                           1
Gillian (Cain) Walker                  1
Reginald Dorcely                       1
Clyde Lewis     

In [82]:
CUNY_students.duplicated().sum()

4

In [36]:
CUNY_students.drop_duplicates(inplace=True)
CUNY_students.shape

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_32653/2049067064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY_students.drop_duplicates(inplace=True)


(39, 9)

In [81]:
CUNY_students['Work Titles']
# CUNY_students['URL'][0]

0      ['\nAdjunct Assistant ProfessorAdjunct Assista...
5      ['\nData Integration ManagerData Integration M...
10     ['\nProject Engineer/Resident EngineerProject ...
21     ['\nInstructor and CSTEP Undergraduate Student...
32     ['\nFull Time Core Mathematics InstructorFull ...
45                                                    []
50     ['\nData SpecialistData Specialist\n', '\nRese...
53     ['\nData EngineerData Engineer\n', '\nSenior D...
54     ['\nIT Support SpecialistIT Support Specialist...
55     ['\nResearch AssociateResearch Associate\n', '...
65          ['\nAssociate DirectorAssociate Director\n']
70              ['\nMedical DirectorMedical Director\n']
78     ['\nMedical Science LiaisonMedical Science Lia...
89     ['\nAssistant Director, NYS- Collegiate Scienc...
97     ['\n Academic Medicine Hospitalist  Academic M...
98     ['\nAdjunct FacultyAdjunct Faculty\n', '\nAdju...
100    ['\nPolicy ManagerPolicy Manager\n', '\nPolicy...
114    ['\nEnvironmental Scient

In [80]:
def clean_titles(A):
    B=[]
    for i in A:
        j=''.join(re.findall('[a-zA-Z]+',i[2:-2]))
        print(j)
        B.append(j)
    return B
clean_titles(['\\nAdjunct Assistant ProfessorAdjunct Assistant Professor\\n', '\\nAdjunctAdjunct\\n', '\\nPre-doctoral Research FellowPre-doctoral Research Fellow\\n', '\\nHealth EducatorHealth Educator\\n', '\\nCommunity Health Research Coordinator / Health EducatorCommunity Health Research Coordinator / Health Educator\\n'])

AdjunctAssistantProfessorAdjunctAssistantProfessor
AdjunctAdjunct
PredoctoralResearchFellowPredoctoralResearchFellow
HealthEducatorHealthEducator
CommunityHealthResearchCoordinatorHealthEducatorCommunityHealthResearchCoordinatorHealthEducator


['AdjunctAssistantProfessorAdjunctAssistantProfessor',
 'AdjunctAdjunct',
 'PredoctoralResearchFellowPredoctoralResearchFellow',
 'HealthEducatorHealthEducator',
 'CommunityHealthResearchCoordinatorHealthEducatorCommunityHealthResearchCoordinatorHealthEducator']